In [1]:
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.getcwd(),"../Algorithm encapsulation"))
sys.path.append(os.path.join(os.getcwd(),".."))
print(sys.path)
from LMPband import LMPband
import pandas as pd
from ga_platoon import max_dt

['f:\\workspace\\jupyter\\论文\\graduation thesis\\Case Analysis', 'd:\\anaconda\\envs\\cplex\\python37.zip', 'd:\\anaconda\\envs\\cplex\\DLLs', 'd:\\anaconda\\envs\\cplex\\lib', 'd:\\anaconda\\envs\\cplex', '', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\chardet-3.0.4-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\docloud-1.0.375-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\urllib3-1.25.11-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\fonttools-4.36.0-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\cvxopt-1.3.0-py3.7-win-amd64.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\pytz-2022.2.1-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\win32', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\win32\\lib', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\Pythonwin', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\0\\.ipython', 'f:\\wor

In [2]:
def get_rf(d,p):
    tmp=[]
    for i,a in enumerate(p):
        idx=np.where(a!=0)[0]
        a=a&0
        if len(idx)>0:
            a[:idx[0]]=1
        tmp.append((d[i]*a).sum())
    return tmp

In [3]:
phase = np.loadtxt("./data/phase")
cir=phase[0].sum()
phase=phase/cir
vol = np.loadtxt("./data/volume")
qv = np.loadtxt("./data/path_vol")
phase_get = np.load("./data/phase_get.npy")
d = np.loadtxt("./data/distance")
straight_get = np.load("./data/straight_get.npy")
ison = np.array([0, 0, 0, 1, 1, 1])
px = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
pxb = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
qb=[120,120]
lower, upper = 8, 14
linspace = np.arange(lower, upper, 1)
lin_num = len(linspace)

# 南北直行方向绿灯时间
sg=np.array([(straight_get[i]*phase).sum(axis=1) for i in range(len(straight_get))])
rho = vol[0] / vol[1]
num = len(vol[0])
numr = len(qv)
nump = len(phase[0])
M = 1e6
nx = 0.00000001
be = 8
speedScope = np.array([lower, upper])
speedVar = np.array([-2, 2])

g=np.array([(phase_get[i]*phase).sum(axis=1) for i in range(numr) ])
r=1-g
rf=np.array([get_rf(phase,phase_get[i]) for i in range(numr)])

ex=[1,0,1,0,1,0,1,1,1,1,1,0]
dwt=15
qb_x=np.ones_like(sg)*20
cap=np.ones_like(sg,dtype=int)*3

spd_on=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","上行速度数据.csv"),index_col=0)
spd_in=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","下行速度数据.csv"),index_col=0)

In [4]:
lmpb=LMPband(phase, [80,180], vol, qv, phase_get, d, straight_get, ison, px, pxb, qb,qb_x,cap,ex, dwt, 
           linspace, be, speedScope,speedVar,speedScope,speedVar,spd_on,spd_in)

lmpb._M1_solve()


conflict(s): 0
None
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective -13910.2824.
Tried aggregator 3 times.
MIP Presolve eliminated 9444 rows and 434 columns.
MIP Presolve modified 1382 coefficients.
Aggregator did 826 substitutions.
Reduced MIP has 1104 rows, 580 columns, and 3930 nonzeros.
Reduced MIP has 147 binaries, 76 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (20.80 ticks)
Probing fixed 2 vars, tightened 64 bounds.
Probing time = 0.00 sec. (0.39 ticks)
Cover probing fixed 0 vars, tightened 4 bounds.
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 23 rows and 17 columns.
MIP Presolve modified 285 coefficients.
Reduced MIP has 1081 rows, 563 columns, and 3859 nonzeros.
Reduced MIP has 143 binaries, 76 generals, 0 SOSs, and 0 indica

In [5]:
lmpb._M2_solve()

[[array([0.0199, 0.0795, 0.1561, 0.151 , 0.072 , 0.0169]), array([0.0003, 0.0051, 0.0384, 0.1265, 0.1814, 0.1134]), array([0.0009, 0.0029, 0.0082, 0.0192, 0.0371, 0.0594])], [array([0.0107, 0.0586, 0.1469, 0.1691, 0.0894, 0.0217]), array([0.0031, 0.0381, 0.153 , 0.203 , 0.089 , 0.0129]), array([0.0002, 0.0041, 0.0387, 0.1404, 0.1935, 0.1014])]]
conflict(s): 0
None
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125

Multi-objective solve log . . .

Index  Priority  Blend          Objective      Nodes  Time (sec.)  DetTime (ticks)
    1         2      1   1.8483886027e+04       1962         1.53           421.93

Performing restart 1
    2         1      1   5.0465739980e+01       8710         4.75          2674.79
status  = multi-objective optimal
time    = 6.296 s.
problem = MILP
gap     = 5.41012e+72%

object value 18484


In [6]:
a=lmpb.get_dataframe()
a

,cross_number,distance,b1,b2,b3,b4,b5,b6,offset,p,...,u3,u4,u5,u6,bb1,bb2,dw1,dw2,tb1,tb2
0,1.0,800.0,23.601259,23.601260,0.000000,23.127776,0.000000,31.803115,0.000000,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,23.601259,31.803115,15.0,15.0,83.692410,72.142857
1,2.0,520.0,21.760027,21.760026,0.000000,30.798807,0.000000,30.798807,83.692409,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,21.760027,30.798807,15.0,15.0,48.685781,53.753729
2,3.0,500.0,21.760026,21.760027,0.000000,19.249254,0.000000,19.249254,49.606397,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,21.760027,19.249254,15.0,15.0,50.714286,53.273533
3,4.0,490.0,21.341564,21.341564,0.000000,19.249254,0.000000,19.249254,38.388299,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,21.341564,19.249254,15.0,15.0,35.000000,48.901641
4,5.0,370.0,21.341564,21.341564,0.000000,21.341564,0.000000,21.341564,30.495939,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,21.341564,21.341564,15.0,15.0,41.428571,41.428571
5,6.0,254.0,19.893324,17.601088,19.893324,53.212596,0.000000,53.212596,81.865532,1.0,...,1.000000e-08,1.000000e-08,1.000000e-08,2.083779e-02,19.893324,53.212596,15.0,15.0,31.750000,31.750000
6,7.0,585.0,19.893324,17.601091,19.893324,16.481532,0.000000,19.893324,47.468182,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,19.893324,16.481532,15.0,15.0,56.785714,56.785714
7,8.0,1020.0,17.139375,0.000000,17.139375,17.139375,25.968750,0.000000,35.758968,1.0,...,1.000000e-08,1.000000e-08,2.075000e-01,1.000000e-08,17.139375,17.139375,15.0,15.0,87.857143,119.892857
8,9.0,409.0,30.954750,0.000000,30.954750,17.139375,25.968750,0.000000,12.833424,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,30.954750,17.139375,15.0,15.0,51.125000,48.543062
9,10.0,547.0,39.368625,0.000000,39.368625,27.704751,27.704750,0.000000,59.751486,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,39.368625,30.954750,15.0,15.0,54.071429,54.071429


In [7]:
lmpb.get_fphase()

array([[  0.        ,  23.60125953,  23.60125953,  55.40437521,
          0.        ,  55.40437521,  68.62777594,  68.62777594,
         83.69240968],
       [  0.        ,  35.65296652,  35.65296652,  66.45177328,
          0.        ,  66.45177328,  66.45177328,  83.69240968,
         83.69240968],
       [  0.        ,  21.76002652,  21.76002652,  41.00928074,
          0.        ,  48.54159761,  66.95392774,  66.95392774,
         83.69240968],
       [ 60.5096122 ,  60.5096122 ,  83.69240968,  83.69240968,
          0.        ,  21.25787206,  42.51574412,  42.51574412,
         60.5096122 ],
       [ 21.34156447,  21.34156447,  42.68312894,  42.68312894,
          9.79201193,  42.68312894,  42.68312894,  83.69240968,
          0.        ],
       [  0.        ,   0.        ,  53.21259592,  53.21259592,
          0.        ,  53.21259592,  53.21259592,  81.86553219,
         81.86553219],
       [ 17.60108942,  17.60108942,  37.49441374,  37.49441374,
          0.        ,  37.4944

In [8]:
from  SimAnalysis import SimAnalysis
df=lmpb.get_dataframe()
fg=lmpb.get_fphase()
sim=SimAnalysis(df,phase,ex,os.path.join(os.getcwd(),"..","vissim","aa","zhongsan.inp"),
               os.path.join(os.getcwd(),"..","vissim","aa","vissim.ini"),
               os.path.join(os.getcwd(),"..","vissim","aa","zhongsan - 副本.inp"),
               fg)
sim.start()
spdx=sim.analysis_speed(os.path.join(os.getcwd(),"..","vissim","aa","zhongsan - 副本.fzp"))
delayx=sim.analysis_delay(os.path.join(os.getcwd(),"..","vissim","aa",'zhongsan - 副本.vlz'))
tvltx=sim.analysis_traveltime(os.path.join(os.getcwd(),"..","vissim","aa",'zhongsan - 副本.rsz'))

[  0.        0.       31.1625   31.1625   31.1625   50.48325  79.56825
  79.56825 103.875  ]
[ 29.5005    29.5005    68.869125  68.869125  68.869125   0.
   0.         0.       103.875   ]
[  0.         0.        50.89875   50.89875   50.89875   75.309375
  89.020875  89.020875 103.875   ]
[  0.         0.        39.368625  39.368625  39.368625  74.3745
 103.875    103.875    103.875   ]
[  0.        0.       30.95475  30.95475  30.95475  66.27225  66.27225
 103.875   103.875  ]
[  0.         0.        17.139375  17.139375  17.139375  43.108125
  70.738875  70.738875 103.875   ]
[17.60108942 17.60108942 37.49441374 37.49441374  0.         37.49441374
 48.87372271 69.91316449 81.86553219]
[ 0.          0.         53.21259592 53.21259592  0.         53.21259592
 53.21259592 81.86553219 81.86553219]
[21.34156447 21.34156447 42.68312894 42.68312894  9.79201193 42.68312894
 42.68312894 83.69240968  0.        ]
[60.5096122  60.5096122  83.69240968 83.69240968  0.         21.25787206
 42.5157

In [9]:
delayx

,timeNo.:,delay1,stoped1,stops1,vehicle1,per delay1,pers1,delay2,stoped2,stops2,...,stops7,vehicle7,per delay7,pers7,delay8,stoped8,stops8,vehicle8,per delay8,pers8
0,600,0.0,0.0,0.00,0,0.0,0,108.6,82.4,4.33,...,0.00,0,0.0,0,0.0,0.0,0.00,0,0.0,0
1,1200,261.0,178.8,10.00,1,261.0,1,238.0,169.4,9.08,...,10.00,1,261.0,1,272.9,161.9,11.33,3,279.0,2
2,1800,354.0,250.5,12.42,26,388.3,50,223.2,150.4,8.33,...,12.42,26,388.3,50,318.0,194.4,16.14,7,359.5,5
3,2400,381.5,255.2,12.86,14,388.8,62,297.7,215.4,12.62,...,12.86,14,388.8,62,286.1,165.6,14.38,8,294.9,7
4,3000,419.8,289.1,15.08,24,443.2,72,328.0,235.4,16.17,...,15.08,24,443.2,72,368.4,212.7,34.00,14,391.8,12
5,3600,457.0,313.4,23.55,29,441.3,53,347.5,227.4,24.90,...,23.55,29,441.3,53,314.9,184.3,25.33,6,385.5,4
6,Total,405.7,279.7,16.57,94,416.3,238,282.5,196.7,13.88,...,16.57,94,416.3,238,325.8,190.9,23.42,38,355.4,30
